In [2]:
# setup the matplotlib graphics library and configure it to show figures inline in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
# asking Python to get all of the basic functions (this is what * means) from the NumPy module.
from numpy import *
# make qutip available in the rest of the notebook
from qutip import *

Resources used: 
- https://arxiv.org/pdf/1909.13651.pdf (Section II).
- https://qutip.org/docs/latest/apidoc/functions.html

## 1. Important polarization states 
- Photon polarization states are represented by vectors in an 2-dimensional space. 
- Quantum states are represented by an object. Other pre-defeined functions can operate on these objects, and we can define our own operations.
- The most fundamental object in QuTiP is the Qobj. They are a matrix (or vector) implemented efficiently in Python. The features most relevant to this work are the indication of their status as either bra or ket, and whether they are Hermitian operators or not.
- Note: It's important to be aware of the basis any time a vector or matrix is used to represent a quantum state.

### Column vectors corresponding to each polarization state
(= basis states for the three standard photon polarizations)

= Element n°0 of our setup!!

In [4]:
# Horizontal polarization state in HV basis (ket)
H = basis(2,0)
H

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

In [5]:
# Vertical polarization state in HV basis (ket)
V = basis(2,1)
V

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]

In [6]:
# +45 polarization state in HV basis (ket)
p45 = 1/sqrt(2)*(H + V)
p45

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

In [7]:
# -45 polarization state in HV basis (ket)
n45 = 1/sqrt(2)*(H - V)
n45

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.70710678]
 [-0.70710678]]

In [8]:
# Left polarization state in HV basis (ket)
L = 1/sqrt(2)*(H + 1j*V)
L

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678+0.j        ]
 [0.        +0.70710678j]]

In [9]:
# Right polarization state in HV basis (ket)
R = 1/sqrt(2)*(H - 1j*V)
R

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678+0.j        ]
 [0.        -0.70710678j]]

### Playing around with the defined states

In [10]:
# Checking that the HV basis is indeed orthogonal (inner product should return 0):
# Turn H into a bra with .dag()
H.dag()*V
V.dag()*H

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]

## 2. Important optical elements

### Matrices corresponding to each operator
- There are several elements that we won't need for our simulation but I put all the basic operators. 

- Can make futher use of the .dag() method to create projection operators, which can be used to project a state into either horiziontal or vertical polarization. In this way, they represent physical operation of passing light through a polarizer.

In [11]:
# Horizontal Polarizer 
PH = H*H.dag()
PH

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [12]:
# Vertical Polarizer 
PV = V*V.dag()
PV

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

- Alternatively, can make use of the QuTiP function ket2dm() or bra2dm(). It takes as input a ket or bra vector and returns the density matrix formed by the outer product.
https://qutip.org/docs/latest/apidoc/functions.html

In [13]:
ket2dm(H)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

- Another option still, is to manually enter the Jones matrices of the polarizers. 

In [14]:
# Horizontal Polarizer 
Qobj([[1,0], [0,0]]) 

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [15]:
# Vertical Polarizer 
Qobj([[0,0], [0,1]]) 

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

In [16]:
# Linear Polarizer, transmission axis θ wrt horizontal 
θ = 45
Jθ = Qobj([[cos(θ)**2,cos(θ)*sin(θ)], [cos(θ)*sin(θ),sin(θ)**2]]) 
Jθ

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.27596319 0.44699833]
 [0.44699833 0.72403681]]

In [17]:
# Quarter-wave plate, fast axis at +/- 45°
a = -1 # +45° -> -1, -45° -> +1
Jλ_4_45 = Qobj([[1/sqrt(2),a*1j*1/sqrt(2)], [a*1j*1/sqrt(2),1/sqrt(2)]]) 
Jλ_4_45

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.70710678+0.j         0.        -0.70710678j]
 [0.        -0.70710678j 0.70710678+0.j        ]]

In [18]:
# Quarter-wave plate, fast axis θ wrt horizontal
θ = 45
Jλ_4_θ = Qobj([[cos(θ)**2 +1j*sin(θ)**2,(1-1j)*sin(θ)*cos(θ)], [(1-1j)*sin(θ)*cos(θ),sin(θ)**2 +1j*cos(θ)**2]]) 
Jλ_4_θ

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.27596319+0.72403681j 0.44699833-0.44699833j]
 [0.44699833-0.44699833j 0.72403681+0.27596319j]]

In [19]:
# Half-wave plate, fast axis θ wrt horizontal
# = Element n°2 of our setup!! with θ = 45°
θ = 45
Jλ_2_θ = Qobj([[cos(2*θ),sin(2*θ)], [sin(2*θ),-cos(2*θ)]]) 
Jλ_2_θ

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[-0.44807362  0.89399666]
 [ 0.89399666  0.44807362]]

In [20]:
# Polarization rotation matrix (this operator corresponds to the rotation of a polarization state by a given angle)
def Rp(θ):
    M = [[cos(θ),-sin(θ)],[sin(θ),cos(θ)]]
    return Qobj(M).tidyup()
                            # returns a Qobj that has been processed with the .tidyup() which removes 
                            # any very small elements from the returned Qobj.
                            # often useful as numerical artifacts from the finite precision of the computer 
                            # may accumulate if operators are used repeatedly.

Rp(45)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.52532199 -0.85090352]
 [ 0.85090352  0.52532199]]

### Simple calculation example
Calculate PV|ψ+45> which represents the photon state that would result from passing light polarized at +45° through a vertical polarizer.

In [21]:
# Define state to be the +45 polarization state in HV basis
psi = p45
# Operating on psi with PV
PV*psi 

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]]

### Polarizing Beam splitter 
= Polarization analyzer PA = beam displacing polarizer (can both split a beam into two components and displace each of these components differently). 

= Element n°1,3,4 of our setup!! (PA_HV and PA_45)

- Can look into these for inspiration:
    - https://physics.stackexchange.com/questions/411560/creating-an-operator-for-a-polarizing-beam-splitter
    - https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_%28Physical_and_Theoretical_Chemistry%29/Quantum_Tutorials_%28Rioux%29/Quantum_Optics/275%3A_Quantum_Seeing_in_the_Dark_-_A_Matrix-Tensor_Analysis
    - https://copilot.caltech.edu/documents/16791/weihs_zeillinger_photon_statistics_at_beamsplitters_qip.pdf (p.9-11).